In [44]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "0"
os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

import random
import torch
import torch.nn as NN
from torch.nn import functional as F
import matplotlib.pyplot as plt

with open('data/cubexfiles.txt', 'r', encoding='utf-8') as f:
    text = f.read()

N = 1000

In [45]:
import bpeasy
from bpeasy.tokenizer import BPEasyTokenizer

gpt4_regex = r"""(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+"""
vocab = bpeasy.train_bpe(iter([text]), gpt4_regex, 16, N)
special_tokens = []
tokenizer = BPEasyTokenizer(vocab, gpt4_regex, [], fill_to_nearest_multiple_of_eight=True, name="skype_msgs_tok")

encode = lambda s: tokenizer.encode(s)
decode = lambda l: tokenizer.decode(l)
text_enc = encode(text)
LEN = len(text_enc)
CTX_SZ = 128 #8
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [51]:
sorted([v.decode('utf-8', 'replace') for v in vocab], key=lambda s: -len(s))

['........',
 ' человек',
 ' которые',
 ' вопрос',
 ' просто',
 'аглянув',
 '<ССЫЛКА',
 ' информ',
 ' только',
 '<ЦИТАТА',
 ' огонё',
 'КАРТИН',
 ' будет',
 ' чтобы',
 ' челов',
 ' время',
 'уществ',
 ' может',
 ' форум',
 'ССЫЛКА',
 ' пишет',
 ' очень',
 ' котор',
 ' получ',
 ' можно',
 ' много',
 ' говор',
 ' этого',
 'ЦИТАТА',
 ' Бабик',
 ' когда',
 ' работ',
 'ность',
 'ствен',
 'опрос',
 ' врем',
 ' тоже',
 'ейств',
 'ления',
 'ается',
 ' сказ',
 ' меня',
 'ическ',
 'итель',
 ' даже',
 ' друг',
 ' посл',
 'олько',
 ' этом',
 'ением',
 'иться',
 'терес',
 ' прав',
 ' долж',
 ' быть',
 'ности',
 ' перв',
 ' пост',
 ' огон',
 ' себя',
 'ществ',
 ' того',
 ' если',
 ' боль',
 'аться',
 'атель',
 ' пред',
 ' есть',
 ' случ',
 ' всех',
 ' было',
 ' мест',
 ' Зем',
 ' год',
 ' том',
 ' тех',
 ' чем',
 ' теб',
 '....',
 '...\n',
 ' отк',
 'chel',
 ' при',
 ' исп',
 ' ещё',
 ' пер',
 'огда',
 'овор',
 'осто',
 'орум',
 'олог',
 ' рас',
 'ольз',
 ' чер',
 ' пон',
 ' про',
 ' мне',
 ' выс',


In [4]:
xs, ys = [], []
for s in range(LEN - CTX_SZ):
    xs.append(text_enc[s:s+CTX_SZ])
    ys.append(text_enc[s+1:s+CTX_SZ+1])

tmp = list(zip(xs, ys))
random.shuffle(tmp)
xs, ys = zip(*tmp)
xs, ys = list(xs), list(ys)

n = int(0.9 * LEN)
xs_trn, ys_trn = torch.tensor(xs[:n], dtype=torch.int64), torch.tensor(ys[:n], dtype=torch.int64)
xs_val, ys_val = torch.tensor(xs[n:], dtype=torch.int64), torch.tensor(ys[n:], dtype=torch.int64)

xs_trn, ys_trn = xs_trn.to(device), ys_trn.to(device)
xs_val, ys_val = xs_val.to(device), ys_val.to(device)

In [52]:
class SelfAttentionHead(NN.Module):
    def __init__(self, head_sz, n_emb, ctx_sz, dropout):
        super().__init__()
        self.keys = NN.Linear(n_emb, head_sz, bias=False)
        self.queries = NN.Linear(n_emb, head_sz, bias=False)
        self.values = NN.Linear(n_emb, head_sz, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(ctx_sz, ctx_sz)))
        self.dropout = NN.Dropout(dropout)
    def forward(self, x):
        B, T, C = x.shape
        k = self.keys(x) #    (B,T,C)
        q = self.queries(x) # (B,T,C)
        weights = q @ k.transpose(-2, -1) * C**-0.5 # (B,T,T)
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)
        weights = F.softmax(weights, dim=-1) # (B,T,T)
        weights = self.dropout(weights)
        v = self.values(x) # (B,T,C)
        out = weights @ v # (B,T,C)
        return out

class SelfAttentionMultiHead(NN.Module):
    def __init__(self, n_heads, head_sz, n_emb, ctx_sz, dropout):
        super().__init__()
        self.heads = NN.ModuleList([SelfAttentionHead(head_sz, n_emb, ctx_sz, dropout) for _ in range(n_heads)])
        self.proj = NN.Linear(n_emb, n_emb)
        self.dropout = NN.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(NN.Module):
    def __init__(self, n_emb, multiplier, dropout):
        super().__init__()
        self.net = NN.Sequential(
            NN.Linear(n_emb, n_emb * multiplier),
            NN.ReLU(),
            NN.Linear(multiplier * n_emb, n_emb),
            NN.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class TransformerBlock(NN.Module):
    def __init__(self, n_heads, n_emb, ctx_sz, dropout):
        super().__init__()
        head_sz = n_emb // n_heads
        self.sa = SelfAttentionMultiHead(n_heads, head_sz, n_emb, ctx_sz, dropout)
        self.ffwd = FeedForward(n_emb, 4, dropout)
        self.ln1 = NN.LayerNorm(n_emb)
        self.ln2 = NN.LayerNorm(n_emb)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class TransformerLanguageModel(NN.Module):
    def __init__(self, n_blocks, n_heads, voc_sz, n_emb, ctx_sz, dropout):
        super().__init__()
        self.ctx_sz = ctx_sz
        self.tok_emb_table = NN.Embedding(voc_sz, n_emb)
        self.pos_emb_table = NN.Embedding(ctx_sz, n_emb)
        self.blocks = NN.Sequential(*(
            ([TransformerBlock(n_heads, n_emb, ctx_sz, dropout)] * n_blocks) +
            [NN.LayerNorm(n_emb)]
        ))
        self.lm_head = NN.Linear(n_emb, voc_sz)
    def forward(self, idx, targets=None):
        B, T = idx.shape        
        tok_emb = self.tok_emb_table(idx)
        pos_emb = self.pos_emb_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            loss = F.cross_entropy(logits, targets.view(B * T))        
        return logits, loss
    def generate(self, idx, max_new_tok):
        for _ in range(max_new_tok):
            idx_crop = idx[:, -self.ctx_sz:]
            logits, _ = self(idx_crop)
            probs = F.softmax(logits[:, -1, :], dim=1)
            topk_probs, topk_indices = torch.topk(probs, 16, dim=-1)
            idx_nxt = torch.multinomial(topk_probs, num_samples=1)
            xcol = torch.gather(topk_indices, -1, idx_nxt)
            idx = torch.cat((idx, xcol), dim=1)
        return idx

In [53]:
N_EMB = 512 #384 #32
N_HEADS = 8
N_BLOCKS = 8
EVAL_INT = 10
model = TransformerLanguageModel(N_BLOCKS, N_HEADS, N, N_EMB, CTX_SZ, 0.2).to(device)

In [5]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

@torch.no_grad()
def est_loss(model, d):
    out = {}
    model.eval()
    for splt in ['trn', 'val']:
        lossi = torch.zeros(EVAL_INT)
        for k in range(EVAL_INT):
            bix = torch.randint(d[splt][0].shape[0] - 1, (BATCH_SZ,)).to(device)
            _, loss = model(d[splt][0][bix], d[splt][1][bix])
            lossi[k] = loss.item()
        out[splt] = lossi.mean()
    model.train()
    return out

BATCH_SZ = 96 #32
#lossi = {'trn':[], 'val':[]}
#for i in range(3500):
#    xb, yb = xs_trn[i*BATCH_SZ : (i+1)*BATCH_SZ], ys_trn[i*BATCH_SZ : (i+1)*BATCH_SZ]
#    logits, loss = model(xb, yb)
#    optimizer.zero_grad(set_to_none=True)
#    loss.backward()
#    optimizer.step()
#    print(i,end=',')
#    if i % 500 == 0:
#        est = est_loss(model, {'trn': [xs_trn, ys_trn], 'val': [xs_val, ys_val]})
#        lossi['trn'].append(est['trn'])
#        lossi['val'].append(est['val'])
#        print(est['trn'])
#plt.plot(lossi['trn'])
#plt.plot(lossi['val'])

<All keys matched successfully>

In [54]:
model.load_state_dict(torch.load('./cbx777.m'))

<All keys matched successfully>

In [7]:
import gc
with torch.no_grad():
    model = None
    gc.collect()
    torch.cuda.empty_cache()

In [11]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 11.576MB


In [11]:
xs_trn.shape

torch.Size([480148, 256])

In [42]:
import numpy as np
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)
len(text)
device

4209640


'cuda'

In [55]:
print(decode(model.generate(torch.zeros((1,1), dtype=torch.long).to(device), max_new_tok=1000)[0].tolist())) 

 евает, и водину, а не у них на пустоте. Потом на столе и пирамида, у них там прямое звёздное в тексте.
	Но если ты вместо этого рулевого метафона, сделана редко установленные домой молитвы, а несколькие и растацы. Так и короче, ведь я, какая утверждает, что идется потеряли внешние детектория событий и территории Хутора, сноватое к нему не было. Индусский язык, услышав из себя, что это был торгов. В неболазлом были сердца, а не изображались, но не знал Запада, как исчасть несущие, а на незамениваются наглый мертвый разумного, поэмоциональный психологический альтернатива из Тарты. По содержаниющие документов, американцы подтвердили этих коллекоп Amplass, Vet - масштаб, мифологию, середи дни и т.п.
	Ученые устрашнюю голову приходящих, прямую секунду, в каждой X. WPI). М. в., "помографической" неизбавляются отсчитания карманам, в которая замирает мероящей картинки, но приметырённых на море, прочих привелых и водих, которые в твой раз в молоде, а кровь: пингал.
	А.Г. - Но, контраст и фильм

In [56]:
print(decode(model.generate(torch.tensor([encode("Tim:")], dtype=torch.long).to(device), max_new_tok=10000)[0].tolist())) 

Tim:	werker, это правильная лучшая соседа, которая достигла между собой волхва, поскольку мы не знаем, показавтив "Молчания", как говорится.
	Господи порно существует 90 крупный сердце сильнейшим блют идентиф. В долиней и приспособлены класса и энергии, которая содержит барабанскую систему "двейца", по которым создал на гравитационным космическим двойник. Он ведет вокруг убийства солнца, так и считает, что он в небес. Итак, кажется диапазона, когда по духе.
	Вторым древних проектом и существует, на какие-тресть природы научились на мобильник и устали свои желанияx. Они сделали на небоскретный архин. Она, на этом приведённым меньшей страница, прежним, между собой, как правил название "Пробуждение на достигает, а также и три воды у него. Если бы он спрыганный, я ничего не помню, что добра и не сейчас, не прочитав её прочее, чем выше, чем кого-нить смешнее, но уйти силы, а не приняли? До недолго ли это сделает из боевой простей за развитие?
	Кто такое? Читайцев? За этот ту же, семечи, кот

In [17]:
print([decode([a]) for a in encode('нашей меня и форуме убежало работаю? 5 него этого практического много')])

['на', 'ш', 'ей', ' меня', ' и', ' форум', 'е', ' у', 'б', 'еж', 'ало', ' работ', 'аю', '?', ' ', '5', ' н', 'его', ' этого', ' пр', 'акт', 'ическ', 'ого', ' много']
